[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/bamirandab/nlp-disaster-tweets/blob/main/notebooks/EDA.ipynb)

# Exploratory Data Analysis - EDA

In [3]:
!kaggle competitions download -c nlp-getting-started -p data

## Load Data

In [ ]:
from zipfile import ZipFile
import os

path = os.path.join('../data/nlp-getting-started.zip')

with ZipFile(path, 'r') as zipObj:
#    # Extract all the contents of zip file in current directory
   zipObj.extractall('../data')

In [1]:
import pandas as pd

data_path = '../data/'
train_df = pd.read_csv(data_path + 'train.csv', encoding='utf-8')
test_df = pd.read_csv(data_path + 'test.csv', encoding='utf-8')